# QIIME 2 Tutorial: Machine Learning

This notebook contains materials accompanying the workshop **Microbiome-Based Tools: From Research to Application**. The notebook and corresponding setup script were adapted from the [**Advanced Block Course: Computational Biology**](https://github.com/bokulich-lab/advanced-comp-bio-tutorial.git); all source code is licensed under the Apache License 2.0.

Save your own local copy of this notebook by using `File > Save a copy in Drive`. At some point you may be prompted to trust the notebook. We promise that it is safe 🤞

**Disclaimer:**

The Google Colab notebook environment will interpret any command as Python code by default. If we want to run bash commands we will have to prefix them by `!`. So any command you see with a leading `!` is a bash command and if you wanted to run it in your terminal you would omit the leading `!`. For example, if in the Colab notebook you ran `!wget` you would just run `wget` in your terminal. 

In this notebook we use the `!` prefix because we run all QIIME 2 commands using the [`q2cli`](https://github.com/qiime2/q2cli/) (QIIME 2 command-line interface). However, QIIME 2 also has a python API and a Galaxy interface. You can learn more about these and other QIIME 2 interfaces at https://qiime2.org/.

You can run the entire notebook by selecting `Runtime > Run all` from the menu in Google Colab. Some steps are time-comsuming and the entire notebook may take up to 30-60 minutes, so run the entire notebook now and we will inspect the commands and results as we work through as a class.

## Setup

QIIME 2 is usually installed by following the [official installation instructions](https://docs.qiime2.org/2023.9/install/). However, because we are using Google Colab and there are some caveats to using conda here, we will have to hack around the installation a little. But no worries, we provide a setup script below which does all this work for us. 😌

So let's start by pulling a local copy of the project repository down from GitHub.

In [ ]:
! git clone https://github.com/bokulich-lab/uzh-microbiome-tutorial.git materials
! mkdir /content/prefetch_cache

We will switch to working within the `material` directory for the rest of the notebook.

In [ ]:
%cd materials

Now we are ready to set up our environment. This will take about 10 minutes.
**Note:** This setup is only relevant for Google Colaboratory and will not work on your local machine. Please follow the [official installation instructions](https://docs.qiime2.org/2023.9/install/) for that.

In [ ]:
%run setup_qiime2

And we will use some Python packages below, so let's load these here:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

## Predicting categorical data
### Training/testing classifier

In [ ]:
! qiime sample-classifier classify-samples \
    --i-table data/table.qza \
    --m-metadata-file data/sample_metadata.qzv \
    --m-metadata-column body-site \
    --p-estimator RandomForestClassifier \
    --p-random-state 0 \
    --output-dir rf_classifier

In [ ]:
! qiime metadata tabulate \
    --m-input-file rf_classifier/predictions.qza \
    --o-visualization rf_classifier/predictions.qzv

In [ ]:
! qiime metadata tabulate \
    --m-input-file rf_classifier/probabilities.qza \
    --o-visualization rf_classifier/probabilities.qzv

In [ ]:
! qiime metadata tabulate \
    --m-input-file rf_classifier/test_targets.qza \
    --m-input-file rf_classifier/predictions.qza \
    --o-visualization rf_classifier/test_targets_predictions.qzv

In [ ]:
! qiime metadata tabulate \
    --m-input-file rf_classifier/feature_importance.qza \
    --o-visualization rf_classifier/feature_importance.qzv

### Feature selection

In [ ]:
! qiime sample-classifier classify-samples \
    --i-table data/table.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --m-metadata-column body-site \
    --p-optimize-feature-selection \
    --p-parameter-tuning \
    --p-estimator RandomForestClassifier \
    --p-n-estimators 20 \
    --p-random-state 123 \
    --output-dir rf_opt_classifier

In [ ]:
! qiime feature-table filter-features \
    --i-table data/table.qza \
    --m-metadata-file rf_opt_classifier/feature_importance.qza \
    --o-filtered-table rf_opt_classifier/important_feature_table.qza

In [ ]:
! qiime sample-classifier heatmap \
    --i-table data/table.qza \
    --i-importance rf_opt_classifier/feature_importance.qza \
    --m-sample-metadata-file data/sample_metadata.tsv \
    --m-sample-metadata-column body-site \
    --p-group-samples \
    --p-feature-count 30 \
    --o-filtered-table rf_opt_classifier/important_feature_table_top_30.qza \
    --o-heatmap rf_opt_classifier/important_feature_heatmap.qzv

**Note:** The model we trained here is a toy example containing very few samples from a single study and will probably not be useful for predicting other unknown samples. But if you have samples from one of these body sites, it could be a fun exercise to give it a spin!

## Predicting continuous data

1. Predict on previous moving pictures dataset
2. Predict on ECAM dataset

In [ ]:
! qiime sample-classifier regress-samples \
    --i-table data/table.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --m-metadata-column days-since-experiment-start \
    --output-dir mp_regressor \
    --verbose